<a href="https://colab.research.google.com/github/Amoghkori/steganography/blob/main/video_hide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import keras
import cv2
import math
import sys
from keras.models import Model
from keras.models import load_model
from PIL import Image
import argparse
from keras.preprocessing.image import save_img
from skimage.util.shape import view_as_blocks

In [ ]:
def normalize_batch(imgs):
    return (imgs -  np.array([0.485, 0.456, 0.406])) /np.array([0.229, 0.224, 0.225])

# Denormalize output images                                                        
def denormalize_batch(imgs,should_clip=True):
    imgs= (imgs * np.array([0.229, 0.224, 0.225])) + np.array([0.485, 0.456, 0.406])
    
    if should_clip:
        imgs= np.clip(imgs,0,1)
    return imgs

# Custom block shuffling
def shuffle(im, inverse = False):
  
  # Configure block size, rows and columns
  blk_size=56
  rows=np.uint8(img.shape[0]/blk_size)
  cols=np.uint8(img.shape[1]/blk_size)

  # Create a block view on image
  img_blks=view_as_blocks(im,block_shape=(blk_size,blk_size,3)).squeeze()
  img_shuff=np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)

  # Secret key maps
  map={0:2, 1:0, 2:3, 3:1}
  inv_map = {v: k for k, v in map.items()}

  # Perform block shuffling
  for i in range(0,rows):
    for j in range(0,cols):
     x,y = i*blk_size, j*blk_size
     if(inverse):
      img_shuff[x:x+blk_size, y:y+blk_size] = img_blks[inv_map[i],inv_map[j]]
     else:
      img_shuff[x:x+blk_size, y:y+blk_size] = img_blks[map[i],map[j]]
      
  return img_shuff


In [ ]:
def update_progress(current_frame, total_frames):
    progress=math.ceil((current_frame/total_frames)*100)
    sys.stdout.write('\rProgress: [{0}] {1}%'.format('>'*math.ceil(progress/10), progress))

In [ ]:
model=load_model("/content/drive/MyDrive/models_mix/hide2.h5",compile=False)

In [ ]:
import cv2
vidcap1 = cv2.VideoCapture("/content/drive/MyDrive/secret_video_up.mp4")
vidcap2 = cv2.VideoCapture("/content/drive/MyDrive/cover_video_up.mp4")
from google.colab.patches import cv2_imshow
fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', 'V')
out = cv2.VideoWriter('container_video_out.mp4', fourcc, 20, (224,224))
while(vidcap1.isOpened()):
  ret1,img1=vidcap1.read()
  ret2,img2=vidcap2.read()
  if ret1==True:
    img3=cv2.resize(img1,(224,224)).reshape(1,224,224,3)/255.0
    img4=cv2.resize(img2,(224,224)).reshape(1,224,224,3)/255.0
    secret = img3
    cover = img4
    coverout=model.predict([normalize_batch(secret),normalize_batch(cover)])
       
            # Postprocess secret image output
    coverout = denormalize_batch(coverout)
    coverout=np.squeeze(coverout)*255.0
    coverout=np.uint8(coverout)
    out.write(coverout)
    if cv2.waitKey(1) & 0xFF==ord("q"):
      break
  else:
    break
vidcap1.release()
vidcap2.release()
out.release()
cv2.destroyAllWindows()
